In [69]:
import sys
from pathlib import Path

notebook_path = Path().resolve()
project_root = notebook_path.parent
sys.path.append(str(project_root))
print('Project root added:', project_root)

Project root added: /Users/aleksandr/MMLS/Music_MMLS


In [70]:
import os
import torch

import wandb

from Music_MMLS.data.ldatamodule import MusicDataModule
from Music_MMLS.models.lmodule import MusicModel
import lightning as L


In [71]:
from hydra import initialize, compose
from omegaconf import OmegaConf

with initialize(config_path='../configs', job_name='demo', version_base='1.3'):
    cfg = compose(config_name='config')

print('Hydra Config:\n')
print(OmegaConf.to_yaml(cfg))

Hydra Config:

project:
  name: Music_MMLS
  seed: 42
  device: cuda
dataset:
  size: 500
  data_dir: ../content/sample_data/Data
  clean_dir: ../content/sample_data/Data/all_records
  noise_dir: ../content/sample_data/Data/noise
  test_size: 0.2
model:
  model: UNet
  n_channels: 1
training:
  epochs: 10
  learning_rate: 0.001
  batch_size: 4
  precision: 32
  optimizer: Adam
  criterion: MSE
  scheduler: ''



In [72]:
clean_files = [os.path.join(cfg.dataset.clean_dir, f)
               for f in os.listdir(cfg.dataset.clean_dir) if f.endswith('.wav')]
noise_files = [os.path.join(cfg.dataset.noise_dir, f) 
               for f in os.listdir(cfg.dataset.noise_dir) if f.endswith('.wav')]

print(f'Found {len(clean_files)} clean files and {len(noise_files)} noise files.')

Found 275 clean files and 5000 noise files.


In [73]:
batch_size = cfg.training.batch_size
num_epochs = cfg.training.epochs
lr = cfg.training.learning_rate

music_data_module = MusicDataModule(cfg.dataset, cfg.training.batch_size)
music_model_module = MusicModel(model_config=cfg.model, training_config=cfg.training)

device = torch.device(cfg.project.device if torch.cuda.is_available() else 'cpu')
wandb_logger = wandb.init(
    project=cfg.project.name,
    config=cfg,
    name=cfg.project.name
)

trainer = L.Trainer(
    max_epochs=cfg.training.epochs,
    logger=wandb_logger,
    devices=-1,
    precision=cfg.training.precision
)

print(f'Training on {device} with batch size {batch_size} for {num_epochs} epochs.')

TypeError: __init__() got an unexpected keyword argument 'model_config'

In [ ]:
history = trainer.fit(
    
    datamodule=music_data_module,
    num_epochs=num_epochs,
    save_best_path="../checkpoints/unet_final"
)

print("Training complete. Final training history:")
print(history)

TypeError: fit() got an unexpected keyword argument 'datamodule'